Installation. In Julia REPL this can be done easier using the packaging mode
(typing `]add COBREXA`)

In [1]:
import Pkg
Pkg.add("COBREXA")

using COBREXA

    Updating registry at `~/.julia/registries/General`
    Updating git-repo `https://github.com/JuliaRegistries/General.git`
   Resolving package versions...
  No Changes to `~/local/research/constraint-based-modeling/summerschool-cobrexa-material/Project.toml`
  No Changes to `~/local/research/constraint-based-modeling/summerschool-cobrexa-material/Manifest.toml`


Let's first get a simple model to have a look at

In [2]:
import Downloads
Downloads.download(
    "http://bigg.ucsd.edu/static/models/e_coli_core.json",
    "e_coli_core.json",
)

"e_coli_core.json"

Load the model

In [3]:
ecoli = load_model("e_coli_core.json");

have a look at what the model contains. First, metabolites

In [4]:
metabolites(ecoli)

72-element Vector{String}:
 "glc__D_e"
 "gln__L_c"
 "gln__L_e"
 "glu__L_c"
 "glu__L_e"
 "glx_c"
 "h2o_c"
 "h2o_e"
 "h_c"
 "h_e"
 "icit_c"
 "lac__D_c"
 "lac__D_e"
 ⋮
 "e4p_c"
 "etoh_c"
 "etoh_e"
 "f6p_c"
 "fdp_c"
 "for_c"
 "for_e"
 "fru_e"
 "fum_c"
 "fum_e"
 "g3p_c"
 "g6p_c"

reactions

In [5]:
reactions(ecoli)

95-element Vector{String}:
 "PFK"
 "PFL"
 "PGI"
 "PGK"
 "PGL"
 "ACALD"
 "AKGt2r"
 "PGM"
 "PIt2r"
 "ALCD2x"
 "ACALDt"
 "ACKr"
 "PPC"
 ⋮
 "ICL"
 "LDH_D"
 "MALS"
 "MALt2_2"
 "MDH"
 "ME1"
 "ME2"
 "NADH16"
 "NADTRHD"
 "NH4t"
 "O2t"
 "PDH"

we can have a look at what each reaction does

In [6]:
reaction_stoichiometry(ecoli, "PFK")

Dict{String, Float64} with 5 entries:
  "adp_c" => 1.0
  "atp_c" => -1.0
  "f6p_c" => -1.0
  "fdp_c" => 1.0
  "h_c"   => 1.0

or look at which metabolites belong to which compartments (Julia looping!)

In [7]:
[m => metabolite_compartment(ecoli, m) for m in metabolites(ecoli)]

72-element Vector{Pair{String, String}}:
 "glc__D_e" => "e"
 "gln__L_c" => "c"
 "gln__L_e" => "e"
 "glu__L_c" => "c"
 "glu__L_e" => "e"
    "glx_c" => "c"
    "h2o_c" => "c"
    "h2o_e" => "e"
      "h_c" => "c"
      "h_e" => "e"
   "icit_c" => "c"
 "lac__D_c" => "c"
 "lac__D_e" => "e"
            ⋮
    "e4p_c" => "c"
   "etoh_c" => "c"
   "etoh_e" => "e"
    "f6p_c" => "c"
    "fdp_c" => "c"
    "for_c" => "c"
    "for_e" => "e"
    "fru_e" => "e"
    "fum_c" => "c"
    "fum_e" => "e"
    "g3p_c" => "c"
    "g6p_c" => "c"

or try to get human-readable names from the reactions

In [8]:
sort([r => reaction_name(ecoli, r) for r in reactions(ecoli)])

95-element Vector{Pair{String, String}}:
                    "ACALD" => "Acetaldehyde dehydrogenase (acetylating)"
                   "ACALDt" => "Acetaldehyde reversible transport"
                     "ACKr" => "Acetate kinase"
                   "ACONTa" => "Aconitase (half-reaction A, Citrate hydro-lyase)"
                   "ACONTb" => "Aconitase (half-reaction B, Isocitrate hydro-lyase)"
                    "ACt2r" => "Acetate reversible transport via proton symport"
                     "ADK1" => "Adenylate kinase"
                    "AKGDH" => "2-Oxogluterate dehydrogenase"
                   "AKGt2r" => "2 oxoglutarate reversible transport via symport"
                   "ALCD2x" => "Alcohol dehydrogenase (ethanol)"
                     "ATPM" => "ATP maintenance requirement"
                   "ATPS4r" => "ATP synthase (four protons for one ATP)"
 "BIOMASS_Ecoli_core_w_GAM" => "Biomass Objective Function with GAM"
                            ⋮
                    "PYRt2" => 

this is useful to e.g. quickly look up dehumanized gene IDs

In [9]:
gs = genes(ecoli)
Dict(gene_name.(Ref(ecoli), gs) .=> gs)

Dict{String, String} with 137 entries:
  "glcB" => "b2976"
  "grcA" => "b2579"
  "sucC" => "b0728"
  "sthA" => "b3962"
  "mhpF" => "b0351"
  "adk"  => "b0474"
  "cbdA" => "b0978"
  "gdhA" => "b1761"
  "pfkB" => "b1723"
  "frdD" => "b4151"
  "sgcE" => "b4301"
  "fumA" => "b1612"
  "aceB" => "b4014"
  "tktA" => "b2935"
  "sdhC" => "b0721"
  "malX" => "b1621"
  "glcA" => "b2975"
  "glnQ" => "b0809"
  "nuoK" => "b2279"
  "gltP" => "b4077"
  "pflC" => "b3952"
  "aqpZ" => "b0875"
  "nuoG" => "b2283"
  "sucD" => "b0729"
  "nuoE" => "b2285"
  ⋮      => ⋮

the model's internals are described by a bipartite graph between reactions
and metabolites, commonly stoichiometry

In [10]:
stoichiometry(ecoli)

72×95 SparseArrays.SparseMatrixCSC{Float64, Int64} with 360 stored entries:
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠆⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⢄⠀⠀⠀⠀⠀⠀⠀⠈⠶⠴⡆⠀⠀⠀⠀⠀⠀
⡀⢐⣀⢀⡀⡒⢒⣐⠀⣂⣂⠀⣂⣂⢂⠀⢀⠀⠀⠀⠀⠀⢀⠄⠀⠀⠀⢂⠀⢂⣀⣐⡒⡀⠆⢙⣀⠀⡀⠀
⠀⠀⠀⠀⠀⠀⠁⠀⠀⠀⠀⠀⠀⠰⠀⠀⠀⠀⠀⠀⠀⠀⠀⠠⠀⠀⠀⠀⠀⡀⠀⠀⠀⠀⠈⢑⣀⣀⠀⠀
⠀⠀⠃⠀⠃⠀⠀⠀⠘⠀⡇⠀⠀⠀⠀⠀⢸⠀⠀⠀⠀⠀⠀⠀⠁⠀⠀⠀⠀⠀⡜⠀⡄⣤⢠⠘⠙⢣⡇⠘
⠀⠐⠀⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠄⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠐⠀⠀⠀⠀⠀⠐⠁⠉⠀⠀⠀⠀⠀⠘⠄
⠀⢐⠀⠂⠀⠄⠠⠠⠀⠠⠆⠀⠄⠀⠄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠁⠀⠀⠠⠀⠠⠀⠀⢀⠀⠀⠠⠀⠀⠁
⢀⠐⠀⠨⢀⠁⠈⣈⠀⢁⣁⠀⠀⠀⠀⠈⠀⠀⠀⠀⠀⠀⠀⠀⠀⠄⠀⠁⢀⠀⢊⠉⠀⠀⠀⢀⠀⣀⠀⢀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡈⠀⡀⠆⠀⠆⠀⡀⠀⠀⠀⠀⠀⠀⠀⠈⠀⠀⠆⠀⠀⠀⠀⠀⠀⠀⠀⠀⠆⠀
⠀⠀⠂⠀⡂⠀⠀⠁⠀⠀⠀⠈⠁⠀⠀⠀⠄⠄⢁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠀⠀⠀⠀⠀⠀
⠈⠀⠁⠀⠀⢀⡀⠀⠠⠁⠁⠀⠑⠀⠐⠲⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠂⠀⠂⠀⠀⠀⠀⠀⠀⠊⠀⠀⠀⠈
⠄⠠⢠⠀⠰⠀⠠⠀⠤⠦⠄⠈⠀⠀⠀⠠⠀⠁⠀⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠤⠄⠄⠠⠀⠀⠀⠀⠀
⠂⠐⠀⠀⠐⡠⢐⠘⢃⠒⠂⡀⠄⠀⠀⠐⠀⠀⠀⢀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠒⠀⢀⢀⠀⠀⣀⠀⢀
⠈⠀⠁⠀⡀⠀⠀⠀⠈⠁⠅⠀⠁⠀⢀⠈⠄⠔⠀⠀⠀⠀⠀⠀⠀⠀⠐⠀⠀⠀⠀⠀⠀⠀⠀⠈⠀⠀⠀⠈
⠣⠁⠀⠀⠀⠀⠀⠀⠀⠀⠁⠀⠀⠀⠈⠀⠁⠁⠀⠈⡀⠀⠀⠀⠀⠀⠐⢣⠈⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⡀⠀⠀⠀⠀⠀⠀⠀⠀⡄⠀⠀⠀⠀⠂⠄⠤⠀⠀⠈⠂⠀⠀⠀⠀⠠⠀⠊⠒⣠⠀⠀⠀⠀⠀⠀⠀⠀⠀

(the matrix is likely zoomed out, but otherwise it's a normal matrix)

Let's try to build a tiny custom model
ref: https://lcsb-biocore.github.io/COBREXA.jl/stable/examples/04b_standardmodel_construction/

The model type for "manually constructed" models we call a StandardModel
because it is kinda standard way to handle stuff in cobra community.
Technically we can convert the above model to StandardModel and play with it:

In [11]:
ecoli = convert(StandardModel, ecoli)
ecoli.reactions["PFK"]
ecoli.reactions["PFK"].lb = 0.5

0.5

But we want to make a completely custom model

In [12]:
model = StandardModel("MyModel")

Metabolic model of type StandardModel


Number of reactions: 0
Number of metabolites: 0


let's make some metabolites

In [13]:
a = Metabolite("a", name = "molecule A", formula = "H2O", compartment = "outside")
b = Metabolite("b") #details omitted for demonstration
c = Metabolite("c")

Metabolite.id: c
Metabolite.name: ---
Metabolite.formula: ---
Metabolite.charge: ---
Metabolite.compartment: ---
Metabolite.notes: ---
Metabolite.annotations: ---


Push the prepared metabolites into the model (the ! stands for "execute!", it
is a syntactic convention for warning that the function changes some of the
parameters (the model) in place)

In [14]:
add_metabolites!(model, [a, b, c])

let's make some reactions

In [15]:
b1 = Reaction("b1", lb = 0.0, ub = 10.0);
b1.metabolites = Dict("a" => 1);

b2 = Reaction("b2", lb = 0.0, ub = 10.0);
b2.metabolites = Dict("b" => -1);

b3 = Reaction("b3", lb = 0.0, ub = 10.0);
b3.metabolites = Dict("c" => -1);

add the reactions to the model

In [16]:
add_reactions!(model, [b1, b2, b3])

shortcut for the above, makes an array of 3 reactions at once

In [17]:
list_of_reactions = [Reaction("v$i", lb = 0.0, ub = 10.0) for i = 1:3]

3-element Vector{Reaction}:
 Reaction("v1", nothing, Dict{String, Float64}(), 0.0, 10.0, nothing, nothing, Dict{String, Vector{String}}(), Dict{String, Vector{String}}(), 0.0)
 Reaction("v2", nothing, Dict{String, Float64}(), 0.0, 10.0, nothing, nothing, Dict{String, Vector{String}}(), Dict{String, Vector{String}}(), 0.0)
 Reaction("v3", nothing, Dict{String, Float64}(), 0.0, 10.0, nothing, nothing, Dict{String, Vector{String}}(), Dict{String, Vector{String}}(), 0.0)

...and we can fill them in

In [18]:
list_of_reactions[1].metabolites = Dict("a" => -1, "b" => 1)
list_of_reactions[2].metabolites = Dict("a" => -1, "c" => 1)
list_of_reactions[3].metabolites = Dict("a" => 1, "c" => -1)

# INspecting one of the reactions
rxn = list_of_reactions[1]
rxn

Reaction.id: v1
Reaction.name: ---
Reaction.metabolites: 1.0 a →  1.0 b
Reaction.lb: 0.0
Reaction.ub: 10.0
Reaction.grr: ---
Reaction.subsystem: ---
Reaction.notes: ---
Reaction.annotations: ---
Reaction.objective_coefficient: 0.0


add the reactions to the model

In [19]:
add_reactions!(model, list_of_reactions)

let's have a look at what it looks like as a matrix

In [20]:
stoichiometry(model)

3×6 SparseArrays.SparseMatrixCSC{Float64, Int64} with 9 stored entries:
 1.0    ⋅     ⋅   -1.0  -1.0   1.0
  ⋅   -1.0    ⋅    1.0    ⋅     ⋅ 
  ⋅     ⋅   -1.0    ⋅    1.0  -1.0

Now let's try to numerically solve the model. First, we need a linear solver;
GLPK will do a good job for this purpose. Other alternatives include Tulip
(native interior-point solver), OSQP and Clarabel (for quadratic problems),
SCIP (free and fast), Clp, and very good commercial solvers include Gurobi
and CPLEX (you will need to install the licenses for these manually).

In [21]:
Pkg.add("GLPK")
using GLPK

# Pkg.add("SCIP")
# using SCIP

   Resolving package versions...
  No Changes to `~/local/research/constraint-based-modeling/summerschool-cobrexa-material/Project.toml`
  No Changes to `~/local/research/constraint-based-modeling/summerschool-cobrexa-material/Manifest.toml`


"Solving" the model is finding optimum with respect to some reaction, thus we
first need to choose the objective that the solver should actually optimize.
Here, let's maximize the flux through the reaction "v1".

In [22]:
change_objective!(model, "v1")

You can observe the result in the model reactions' objective coefficients
(and fine-tune that manually if needed):

In [23]:
model.reactions["v1"].objective_coefficient

1.0

Flux balance analysis finds a steady state flux through the model which is
within the reaction flux lower and upper bounds:

In [24]:
solution = flux_balance_analysis(model, GLPK.Optimizer)

A JuMP Model
Maximization problem with:
Variables: 6
Objective function type: JuMP.AffExpr
`JuMP.AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 3 constraints
`JuMP.AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 12 constraints
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: GLPK
Names registered in the model: lbs, mb, ubs, x

The above returned a solved optimization model description. From that we can
extract the actual value of the objective function:

In [25]:
solved_objective_value(solution)

10.0

...and the complete description of the flux through the reactions

In [26]:
flux_vector(model, solution)

6-element Vector{Float64}:
 10.0
 10.0
  0.0
 10.0
  0.0
  0.0

...or a little better as a dictionary

In [27]:
sol_dict = flux_dict(model, solution)

Dict{String, Float64} with 6 entries:
  "v2" => 0.0
  "b3" => 0.0
  "v1" => 10.0
  "b2" => 10.0
  "v3" => 0.0
  "b1" => 10.0

In [28]:
Pkg.add(["DataFrames", "CSV"])
using DataFrames, CSV

   Resolving package versions...
  No Changes to `~/local/research/constraint-based-modeling/summerschool-cobrexa-material/Project.toml`
  No Changes to `~/local/research/constraint-based-modeling/summerschool-cobrexa-material/Manifest.toml`


In [29]:
df = DataFrame(reaction = collect(keys(sol_dict)), flux = collect(values(sol_dict)))
df

Row,reaction,flux
,String,Float64
1,v2,0.0
2,b3,0.0
3,v1,10.0
4,b2,10.0
5,v3,0.0
6,b1,10.0


typically, one does all of this in one step with a shortcut function:

In [30]:
flux = flux_balance_analysis_dict(model, GLPK.Optimizer)

Dict{String, Float64} with 6 entries:
  "v2" => 0.0
  "b3" => 0.0
  "v1" => 10.0
  "b2" => 10.0
  "v3" => 0.0
  "b1" => 10.0

As a check, we can verify that the amount of metabolites in the model indeed
stays balanced; if the solver worked, this vector should be zero (or within
the numerical tolerance of zero):

In [31]:
stoichiometry(model) * flux_vector(model, solution)

3-element Vector{Float64}:
 0.0
 0.0
 0.0

In [32]:
Pkg.add("SCIP")
using SCIP

   Resolving package versions...
  No Changes to `~/local/research/constraint-based-modeling/summerschool-cobrexa-material/Project.toml`
  No Changes to `~/local/research/constraint-based-modeling/summerschool-cobrexa-material/Manifest.toml`


In [33]:
flux = flux_balance_analysis_dict(model, SCIP.Optimizer)

feasible solution found by trivial heuristic after 0.0 seconds, objective value 0.000000e+00
presolving:
(round 1, fast)       1 del vars, 13 del conss, 0 add conss, 10 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 2, fast)       3 del vars, 13 del conss, 0 add conss, 10 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 3, fast)       3 del vars, 13 del conss, 0 add conss, 10 chg bounds, 1 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
   (0.0s) running MILP presolver
   (0.0s) MILP presolver (3 rounds): 1 aggregations, 2 fixings, 0 bound changes
presolving (4 rounds: 4 fast, 1 medium, 1 exhaustive):
 6 deleted vars, 15 deleted constraints, 0 added constraints, 10 tightened bounds, 0 added holes, 1 changed sides, 0 changed coefficients
 0 implications, 0 cliques
transformed 1/2 original solutions to the transformed problem space
Presolving Time: 0.00

SCIP Status        : problem is solved [optimal solution found]
Solving 

Dict{String, Float64} with 6 entries:
  "v2" => 10.0
  "b3" => 0.0
  "v1" => 10.0
  "b2" => 10.0
  "v3" => 10.0
  "b1" => 10.0

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*